<a href="https://colab.research.google.com/github/drdrwenski/sympy-examples/blob/main/MetricTensorEuclideanSpace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metric Tensor in Euclidean Space

The metric tensor is a fundamental concept in the field of differential geometry. It is a mathematical object that encodes the intrinsic geometry of a space, allowing us to measure distances, angles, and volumes within that space.
At its core, the metric tensor is a second-rank tensor that describes the local curvature and distance relationships in a given coordinate system. It is a symmetric, positive-definite matrix that transforms covariantly under changes of the coordinate system. This means that the metric tensor contains all the information needed to perform measurements and calculations within the space, regardless of how the coordinates are defined.

##Preliminaries

First, upgrade to the latest version of SymPy if necessary. This should be at least **v1.10**.

In [2]:
%%capture
%pip install --upgrade sympy

Next, we define some output helper functions, providing shorthand notations for derivatives in curvilinear coordinates (still not working perfectly well).

In [3]:
#@title Output Helper Functions
from IPython.display import display, Markdown
from sympy import latex

def displayLatex(expr, **kwargs):
  intro = kwargs.get('intro', '')
  output = rf'$\displaystyle {intro} {latex(expr)}$'
  display(Markdown(output))

def displayLeibniz(expr,func,sys):
  leibnizComplete = ''
  for term in expr.args:
    leibniz = latex(term)
    # find second derivatives
    for i,j in zip(range(sys.dim),range(sys.dim)):
      coordStr0 = latex(sys.base_scalar(i))
      coordStr1 = latex(sys.base_scalar(j))
      # original notation
      replStr = latex(Differential(Differential(func)(sys.base_vector(i)))(sys.base_vector(j)))
      # shorthand notation
      if i == j:
        subsStr = rf'\frac{{\partial^{{2}} {func.name}}}{{\partial {coordStr0}^{{2}}}}'
      else:
        subsStr = rf'\frac{{\partial^{{2}} {func.name}}}{{\partial {coordStr1} \partial {coordStr0}}}'
      if leibniz.find(replStr) >= 0:
        leibniz = leibniz.replace(replStr,'')
        leibniz = leibniz + subsStr
    # find first derivatives
    for i in range(sys.dim):
      coordStr = latex(sys.base_scalar(i))
      baseStr = latex(sys.base_vector(i))
      # original notation
      replStr = latex(Differential(func)(sys.base_vector(i)))
      # shorthand notation
      subsStr = rf'\frac{{\partial {func.name}}}{{\partial {coordStr}}}'
      if leibniz.find(replStr) >= 0:
        leibniz = leibniz.replace(replStr,'')
        leibniz = leibniz + subsStr
      # rearrange terms
      if leibniz.find(baseStr) >= 0:
        leibniz = leibniz.replace(baseStr,'')
        leibniz = leibniz + baseStr
    # rebuild sum of derivatives
    if len(leibnizComplete) == 0:
      leibnizComplete = leibniz
    else:
      leibnizComplete = leibnizComplete + '+' + leibniz
  leibnizComplete = leibnizComplete.replace(rf'\frac{{ }}',rf'\frac{{1}}')
  leibnizComplete = leibnizComplete.replace(rf'\frac{{}}',rf'\frac{{1}}')
  display(Markdown(rf'$\displaystyle {leibnizComplete}$'))

##Curvilinear Coordinates

Spherical & cylindrical 3D coordinates are introduced utilizing the vector module. The Cartesian coordinate system serves as the parent instance for all other types of coordinate systems since we are considering Euclidean space. Coordinate transformations are then stored as symbolic expressions in a Python dictionary *relations*.
Additional dictionaries provide simplification methods tailored to each coordinate system, enhancing computational efficiency and readability of results.

In [4]:
from sympy.vector import CoordSys3D
Cart = CoordSys3D('Cart')
Spher = Cart.create_new('Spher', transformation='spherical')
Cyl = Cart.create_new('Cyl', transformation='cylindrical')

# Since there is no built-in transformation_to_parent_function() yet,
# we apply the following workaround.
def transformation_to_parent_function(child, scalars):
  sc = (scalars, child.base_scalars())
  abbrev = [(sc[1][i],sc[0][i]) for i in range(3)]
  return tuple(expr.subs(abbrev) for expr in child.transformation_to_parent())

from sympy import symbols, Function, Lambda
from sympy import sqrt
x, y, z = symbols('x y z', real=True)
r, theta, phi = symbols('r theta phi', nonnegative=True)

# Python dictionary of coordinate transformations
relations = {
        ('Cartesian','Spherical'):
            Lambda((x,y,z), Spher.transformation_from_parent_function()(x,y,z)),
        ('Spherical','Cartesian'):
            Lambda((r,theta,phi), transformation_to_parent_function(Spher, (r,theta,phi))),
        ('Cartesian','Cylindrical'):
            Lambda((x,y,z), Cyl.transformation_from_parent_function()(x,y,z)),
        ('Cylindrical','Cartesian'):
            Lambda((r,theta,z), transformation_to_parent_function(Cyl, (r,theta,z)))
}

# Python dictionaries of simplification strategies
from sympy import simplify, expand, factor, cancel
from sympy import expand_power_base, powdenest
from sympy.simplify.fu import fu, TR6
from functools import reduce
from operator import mul

simplifications = {
    'Spherical': (lambda elem: expand(TR6(factor(TR6(elem))))),
    'Cylindrical': (lambda elem: expand(TR6(factor(elem))))
}

powers = {
    'Spherical': (lambda elem: powdenest(expand_power_base(elem, force=True), force=True)),
    'Cylindrical': (lambda elem: powdenest(elem, force=True))
}

We also provide a coordinate transformation from an ellipsoidal coordinate system to a Cartesian coordinate system.

In [ ]:
# Check assumptions:
xi, eta, zeta = symbols('xi eta zeta', real=True)
a, b, c = symbols('a b c', nonnegative=True)

exprs = tuple(
    sqrt(reduce(mul, [(coord**2+ellips) for ellips in (xi,eta,zeta)])/
         reduce(mul, [(cart**2-coord**2) for cart in (a,b,c) if cart != coord]))
    for coord in (a,b,c)
)

relations[('Ellipsoidal','Cartesian')] = Lambda((xi,eta,zeta), exprs)
simplifications['Ellipsoidal'] = (lambda elem: factor(cancel(elem)))
powers['Ellipsoidal'] = (lambda elem: elem)

displayLatex(exprs[0])

$\displaystyle  \sqrt{\frac{\left(a^{2} + \eta\right) \left(a^{2} + \xi\right) \left(a^{2} + \zeta\right)}{\left(- a^{2} + b^{2}\right) \left(- a^{2} + c^{2}\right)}}$

We also provide a coordinate transformation from a prolate spheroidal coordinate system to a Cartesian coordinate system.

In [6]:
from sympy import sin, cos, sinh, cosh
from sympy import Wild
# Check assumptions:
mu, nu, phi = symbols('mu nu phi', nonnegative=True)
a = symbols('a', nonnegative=True)

exprs = (
    a*sinh(mu)*sin(nu)*cos(phi),
    a*sinh(mu)*sin(nu)*sin(phi),
    a*cosh(mu)*cos(nu),
)

def replace_cosh2(expr):
  u = Wild('u')
  return expr.replace(cosh(u)**2, 1+sinh(u)**2)

relations[('ProlateSpheroidal','Cartesian')] = Lambda((mu,nu,phi), exprs)
simplifications['ProlateSpheroidal'] = (lambda elem: factor(expand(replace_cosh2(TR6(elem)))))
powers['ProlateSpheroidal'] = (lambda elem: elem)

displayLatex(exprs, intro = '(x,y,z) = ')

$\displaystyle (x,y,z) =  \left( a \sin{\left(\nu \right)} \cos{\left(\phi \right)} \sinh{\left(\mu \right)}, \  a \sin{\left(\nu \right)} \sin{\left(\phi \right)} \sinh{\left(\mu \right)}, \  a \cos{\left(\nu \right)} \cosh{\left(\mu \right)}\right)$

##Metric Tensor

We set up a 3-dimensional manifold, define a patch on that manifold, and create two coordinate systems on that patch: one Cartesian and one of a user-chosen type ('Spherical', 'Cylindrical' or 'ProlateSpheroidal').

In [9]:
from sympy.diffgeom import Manifold, Patch, CoordSystem, Point
M = Manifold('M', 3)
P = Patch('P', M)

# Try any of 'Spherical', 'Cylindrical', 'ProlateSpheroidal'
CoordSystemType = 'ProlateSpheroidal'
CoordSystemSymbols = relations.get((CoordSystemType,'Cartesian')).args[0]
CoordRelation = dict({
    (CoordSystemType,'Cartesian'): relations.get((CoordSystemType,'Cartesian'))
})

R3 = CoordSystem('Cartesian', P, (x,y,z), CoordRelation)
S0 = CoordSystem(CoordSystemType, P, CoordSystemSymbols, CoordRelation)
nrange = range(S0.dim)

# point = Point(S0, (1,1,1))
# displayLatex(point.coords(R3))

In applications the metric tensor is usually displayed either as a quadratic form $\ ds^2$ or as a matrix $\ (g_{ij})$.

In [10]:
from sympy.matrices import Matrix, diag
from sympy.diffgeom import TensorProduct

Jacob = S0.jacobian(R3, S0.base_scalars())
G_metric = Jacob.T * Jacob
G_metric = G_metric.as_mutable().applyfunc(simplifications.get(CoordSystemType,simplify)).as_immutable()

if G_metric.is_diagonal():
  diagonal = [G_metric[i,i] for i in nrange]
  G_inv = diag(*[1/d for d in diagonal])
  G_sqrt = sqrt(reduce(mul,diagonal))
  G_sqrt = powers.get(CoordSystemType,simplify)(G_sqrt)
else:
  G_inv = G_metric.inv()
  G_sqrt = Jacob.det()

metric_Tensor = sum([ G_metric[i,j] * TensorProduct(S0.base_oneform(i), S0.base_oneform(j))
                      for (i,j) in zip(nrange,nrange) ])

displayLatex(metric_Tensor, intro = 'ds^2 =')
print()
displayLatex(G_metric, intro = '(g_{ij}) = ')
print()
displayLatex(G_inv, intro = '(g^{ij}) = ')
print()
displayLatex(G_sqrt, intro = r'\det (g_{ij}) = ')

$\displaystyle ds^2 = a^{2} \left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right) \operatorname{d}\mu \otimes \operatorname{d}\mu + a^{2} \left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right) \operatorname{d}\nu \otimes \operatorname{d}\nu + a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)} \operatorname{d}\phi \otimes \operatorname{d}\phi$

$\displaystyle (g_{ij}) =  \left[\begin{matrix}a^{2} \left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right) & 0 & 0\\0 & a^{2} \left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right) & 0\\0 & 0 & a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}\end{matrix}\right]$

$\displaystyle (g^{ij}) =  \left[\begin{matrix}\frac{1}{a^{2} \left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right)} & 0 & 0\\0 & \frac{1}{a^{2} \left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right)} & 0\\0 & 0 & \frac{1}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}\end{matrix}\right]$

$\displaystyle \det (g_{ij}) =  a^{3} \sqrt{\left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}$

##Gradient & Laplacian

Next, we define a function that returns the **gradient vector field** of a scalar field *s_field*. It first computes the contravariant components *tmp*, then the vector field is easily obtained by list comprehension.

**Note:** The CoordSystem *sys* and the inverse metric tensor *g_inv* have to be provided by the caller!

In [14]:
from sympy.diffgeom import Differential
func = Function('f')(*S0.base_scalars())

def grad_of_field(s_field, sys, g_inv):
  tmp = g_inv * Matrix([(Differential(s_field)(e)) for e in sys.base_vectors()])
  return sum([tmp[i]*sys.base_vector(i) for i in range(sys.dim)])

gradient = grad_of_field(func, S0, G_inv)
displayLeibniz(gradient, func, S0)
#gradient

$\displaystyle \frac{1}{a^{2} \left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right)}\frac{\partial f}{\partial \mathbf{\mu}}\partial_{\mu}+\frac{1}{a^{2} \left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right)}\frac{\partial f}{\partial \mathbf{\nu}}\partial_{\nu}+\frac{1}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}\frac{\partial f}{\partial \mathbf{\phi}}\partial_{\phi}$

In [20]:
def div_of_field(v_field, sys, g_sqrt):
  tmp = g_sqrt * Matrix([dx(v_field) for dx in sys.base_oneforms()])
  return sum([simplifications.get(str(sys.name),simplify)(Differential(tmp[i])(sys.base_vector(i))/g_sqrt)
              for i in range(sys.dim)])

laplace = powers.get(str(S0.name),simplify)(div_of_field(gradient, S0, G_sqrt)) # .expand().trigsimp()
displayLeibniz(laplace.expand(),func,S0)
# displayLatex(laplace)

$\displaystyle \frac{\cos{\left(\mathbf{\nu} \right)} }{a^{2} \sin^{3}{\left(\mathbf{\nu} \right)} + a^{2} \sin{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}\frac{\partial f}{\partial \mathbf{\nu}}+\frac{\sin{\left(\mathbf{\nu} \right)} }{a^{2} \sin^{3}{\left(\mathbf{\nu} \right)} + a^{2} \sin{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}\frac{\partial^{2} f}{\partial \mathbf{\nu}^{2}}+\frac{\cosh{\left(\mathbf{\mu} \right)} }{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)} + a^{2} \sinh^{3}{\left(\mathbf{\mu} \right)}}\frac{\partial f}{\partial \mathbf{\mu}}+\frac{\sinh{\left(\mathbf{\mu} \right)} }{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)} + a^{2} \sinh^{3}{\left(\mathbf{\mu} \right)}}\frac{\partial^{2} f}{\partial \mathbf{\mu}^{2}}+\frac{1}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}\frac{\partial^{2} f}{\partial \mathbf{\phi}^{2}}$

##Curvature

In [11]:
import pprint
pprint.pp(CoordRelation)

{('ProlateSpheroidal', 'Cartesian'): Lambda((mu, nu, phi), (a*sin(nu)*cos(phi)*sinh(mu), a*sin(nu)*sin(phi)*sinh(mu), a*cos(nu)*cosh(mu)))}


In [12]:
from sympy.diffgeom import Differential

displayLatex(G_metric[0,0])
displayLatex(S0.base_vector(1))

# Differential(G_metric[0,0])(S0.base_vector(1))

$\displaystyle  a^{2} \left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right)$

$\displaystyle  \partial_{\nu}$

In [26]:
#@title Computation of Christoffel symbols
from sympy.diffgeom import Differential
from sympy import Rational

diff_metric = [
    simplifications.get(S0.name,simplify)(Differential(G_metric)(S0.base_vector(k)))
    for k in nrange ]

# note the ordering of the parameters m, i, j
def christoffel_symbol(m,i,j):
  return Rational(1,2)*sum([ G_inv[m,k]*(diff_metric[j][i,k] + diff_metric[i][k,j] - diff_metric[k][i,j]) for k in nrange ])

christoffel = []

for m in nrange:
  upper_triangle = Matrix([
      [powers.get(S0.name,simplify)(christoffel_symbol(m,i,j))
        if i<=j else 0 for j in nrange]
      for i in nrange])
  lower_triangle = Matrix([[upper_triangle[j,i] if i>j else 0 for j in nrange] for i in nrange])
  christoffel.append(upper_triangle + lower_triangle)

displayLatex(christoffel)

$\displaystyle  \left[ \left[\begin{matrix}\frac{\sinh{\left(2 \mathbf{\mu} \right)}}{2 \left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right)} & \frac{\sin{\left(2 \mathbf{\nu} \right)}}{2 \left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right)} & 0\\\frac{\sin{\left(2 \mathbf{\nu} \right)}}{2 \left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right)} & - \frac{\sinh{\left(2 \mathbf{\mu} \right)}}{2 \sin^{2}{\left(\mathbf{\nu} \right)} + 2 \sinh^{2}{\left(\mathbf{\mu} \right)}} & 0\\0 & 0 & - \frac{\sin^{2}{\left(\mathbf{\nu} \right)} \sinh{\left(2 \mathbf{\mu} \right)}}{2 \sin^{2}{\left(\mathbf{\nu} \right)} + 2 \sinh^{2}{\left(\mathbf{\mu} \right)}}\end{matrix}\right], \  \left[\begin{matrix}- \frac{\sin{\left(2 \mathbf{\nu} \right)}}{2 \sin^{2}{\left(\mathbf{\nu} \right)} + 2 \sinh^{2}{\left(\mathbf{\mu} \right)}} & \frac{\sinh{\left(2 \mathbf{\mu} \right)}}{2 \left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right)} & 0\\\frac{\sinh{\left(2 \mathbf{\mu} \right)}}{2 \left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right)} & \frac{\sin{\left(2 \mathbf{\nu} \right)}}{2 \left(\sin^{2}{\left(\mathbf{\nu} \right)} + \sinh^{2}{\left(\mathbf{\mu} \right)}\right)} & 0\\0 & 0 & - \frac{\sin{\left(2 \mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}{2 \sin^{2}{\left(\mathbf{\nu} \right)} + 2 \sinh^{2}{\left(\mathbf{\mu} \right)}}\end{matrix}\right], \  \left[\begin{matrix}0 & 0 & \frac{1}{\tanh{\left(\mathbf{\mu} \right)}}\\0 & 0 & \frac{1}{\tan{\left(\mathbf{\nu} \right)}}\\\frac{1}{\tanh{\left(\mathbf{\mu} \right)}} & \frac{1}{\tan{\left(\mathbf{\nu} \right)}} & 0\end{matrix}\right]\right]$

In [27]:
from sympy.diffgeom import metric_to_Christoffel_2nd
from sympy import srepr

chris = metric_to_Christoffel_2nd(metric_Tensor)
displayLatex(chris)
print(type(chris))
# print(srepr(chris))

$\displaystyle  \left[\begin{matrix}\left[\begin{matrix}\frac{a^{2} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & \frac{a^{2} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & 0\\\frac{a^{2} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & - \frac{a^{2} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & 0\\0 & 0 & - \frac{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}}\end{matrix}\right] & \left[\begin{matrix}- \frac{a^{2} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & \frac{a^{2} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & 0\\\frac{a^{2} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & \frac{a^{2} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & 0\\0 & 0 & - \frac{a^{2} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}}\end{matrix}\right] & \left[\begin{matrix}0 & 0 & \frac{\cosh{\left(\mathbf{\mu} \right)}}{\sinh{\left(\mathbf{\mu} \right)}}\\0 & 0 & \frac{\cos{\left(\mathbf{\nu} \right)}}{\sin{\left(\mathbf{\nu} \right)}}\\\frac{\cosh{\left(\mathbf{\mu} \right)}}{\sinh{\left(\mathbf{\mu} \right)}} & \frac{\cos{\left(\mathbf{\nu} \right)}}{\sin{\left(\mathbf{\nu} \right)}} & 0\end{matrix}\right]\end{matrix}\right]$

<class 'sympy.tensor.array.dense_ndim_array.ImmutableDenseNDimArray'>


In [ ]:
from sympy.diffgeom import metric_to_Riemann_components

Riemann = metric_to_Riemann_components(metric_Tensor)
displayLatex(Riemann)
print(type(Riemann))

$\displaystyle  \left[\begin{matrix}\left[\begin{matrix}0 & 0 & 0\\0 & 0 & 0\\0 & 0 & 0\end{matrix}\right] & \left[\begin{matrix}0 & \frac{2 a^{4} \sin^{2}{\left(\mathbf{\nu} \right)} \cos^{2}{\left(\mathbf{\nu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} + \frac{2 a^{4} \sinh^{2}{\left(\mathbf{\mu} \right)} \cosh^{2}{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} + \frac{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} - \frac{a^{2} \cos^{2}{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} - \frac{a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} - \frac{a^{2} \cosh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & 0\\- \frac{2 a^{4} \sin^{2}{\left(\mathbf{\nu} \right)} \cos^{2}{\left(\mathbf{\nu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} - \frac{2 a^{4} \sinh^{2}{\left(\mathbf{\mu} \right)} \cosh^{2}{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} - \frac{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} + \frac{a^{2} \cos^{2}{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} + \frac{a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} + \frac{a^{2} \cosh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & 0 & 0\\0 & 0 & 0\end{matrix}\right] & \left[\begin{matrix}0 & 0 & - \frac{a^{4} \sin^{2}{\left(\mathbf{\nu} \right)} \cos^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} + \frac{a^{4} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)} \cosh^{2}{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} - \frac{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}}\\0 & 0 & \frac{a^{4} \sin^{3}{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} + \frac{a^{4} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \sinh^{3}{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} - \frac{a^{2} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}}\\\frac{a^{4} \sin^{2}{\left(\mathbf{\nu} \right)} \cos^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} - \frac{a^{4} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)} \cosh^{2}{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} + \frac{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & - \frac{a^{4} \sin^{3}{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} - \frac{a^{4} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \sinh^{3}{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} + \frac{a^{2} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & 0\end{matrix}\right]\\\left[\begin{matrix}0 & - \frac{2 a^{4} \sin^{2}{\left(\mathbf{\nu} \right)} \cos^{2}{\left(\mathbf{\nu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} - \frac{2 a^{4} \sinh^{2}{\left(\mathbf{\mu} \right)} \cosh^{2}{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} - \frac{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} + \frac{a^{2} \cos^{2}{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} + \frac{a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} + \frac{a^{2} \cosh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & 0\\\frac{2 a^{4} \sin^{2}{\left(\mathbf{\nu} \right)} \cos^{2}{\left(\mathbf{\nu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} + \frac{2 a^{4} \sinh^{2}{\left(\mathbf{\mu} \right)} \cosh^{2}{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} + \frac{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} - \frac{a^{2} \cos^{2}{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} - \frac{a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} - \frac{a^{2} \cosh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & 0 & 0\\0 & 0 & 0\end{matrix}\right] & \left[\begin{matrix}0 & 0 & 0\\0 & 0 & 0\\0 & 0 & 0\end{matrix}\right] & \left[\begin{matrix}0 & 0 & \frac{a^{4} \sin^{3}{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} + \frac{a^{4} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \sinh^{3}{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} - \frac{a^{2} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}}\\0 & 0 & \frac{a^{4} \sin^{2}{\left(\mathbf{\nu} \right)} \cos^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} - \frac{a^{4} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)} \cosh^{2}{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} + \frac{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}}\\- \frac{a^{4} \sin^{3}{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} - \frac{a^{4} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \sinh^{3}{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} + \frac{a^{2} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & - \frac{a^{4} \sin^{2}{\left(\mathbf{\nu} \right)} \cos^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} + \frac{a^{4} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)} \cosh^{2}{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right)^{2}} - \frac{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} \sinh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} & 0\end{matrix}\right]\\\left[\begin{matrix}0 & 0 & \frac{a^{2} \cos^{2}{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} - \frac{a^{2} \cosh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} + 1\\0 & 0 & - \frac{a^{2} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right) \sinh{\left(\mathbf{\mu} \right)}} - \frac{a^{2} \cos{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right) \sin{\left(\mathbf{\nu} \right)}} + \frac{\cos{\left(\mathbf{\nu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\sin{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)}}\\- \frac{a^{2} \cos^{2}{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} + \frac{a^{2} \cosh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} - 1 & \frac{a^{2} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right) \sinh{\left(\mathbf{\mu} \right)}} + \frac{a^{2} \cos{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right) \sin{\left(\mathbf{\nu} \right)}} - \frac{\cos{\left(\mathbf{\nu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\sin{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)}} & 0\end{matrix}\right] & \left[\begin{matrix}0 & 0 & - \frac{a^{2} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right) \sinh{\left(\mathbf{\mu} \right)}} - \frac{a^{2} \cos{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right) \sin{\left(\mathbf{\nu} \right)}} + \frac{\cos{\left(\mathbf{\nu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\sin{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)}}\\0 & 0 & - \frac{a^{2} \cos^{2}{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} + \frac{a^{2} \cosh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} - 1\\\frac{a^{2} \sin{\left(\mathbf{\nu} \right)} \cos{\left(\mathbf{\nu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right) \sinh{\left(\mathbf{\mu} \right)}} + \frac{a^{2} \cos{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\left(a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}\right) \sin{\left(\mathbf{\nu} \right)}} - \frac{\cos{\left(\mathbf{\nu} \right)} \cosh{\left(\mathbf{\mu} \right)}}{\sin{\left(\mathbf{\nu} \right)} \sinh{\left(\mathbf{\mu} \right)}} & \frac{a^{2} \cos^{2}{\left(\mathbf{\nu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} - \frac{a^{2} \cosh^{2}{\left(\mathbf{\mu} \right)}}{a^{2} \sin^{2}{\left(\mathbf{\nu} \right)} + a^{2} \sinh^{2}{\left(\mathbf{\mu} \right)}} + 1 & 0\end{matrix}\right] & \left[\begin{matrix}0 & 0 & 0\\0 & 0 & 0\\0 & 0 & 0\end{matrix}\right]\end{matrix}\right]$

<class 'sympy.tensor.array.dense_ndim_array.ImmutableDenseNDimArray'>
